# mohamed yaser elsaid 2001226
# yassa siefen ayed 2001307

In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

## Data preprocessing
firstly we will explore the data and take in considerations columns we nedd and drop others we don't

In [ ]:
df = pd.read_csv("data.csv")
df.head()

### it appears that the unamed is indexing so we will drop it 

In [ ]:
df=df.drop("Unnamed: 0",axis=1)
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

### from describe there are some unusual beahviours observed as :
- the min in the diff coulmn is -ve value
- symptom 6 has its percentiles and max equal to each other
- gender have a max of 2

### fristly we explore the diff as it's described in the project statement that it's the time the symptoms appeared so it can't be less than 0

In [ ]:
print(df[df['diff_sym_hos'] <0])

### next we explore the values of sympotom 6 and the gender

In [ ]:
print(df['symptom6'].value_counts())

#### as can be seen there's only 1 zero so the symptom doesn't effect any model so we will drop it

In [ ]:
gender = df['gender'].value_counts()
print(gender)

#### an undefined variable 2  exists with big frequency so that makes it redundant to make sure we will pie plot the percentages

In [ ]:
labels=[1,0,2] # as one appeared before 0 in the data
plt.pie(gender,labels=labels,startangle=180,autopct="%2.2f%%")

## we can drop symptom 6 as it doesn't affect the data and gender can be dropped but as instructed that the data is clean so we will keep it

In [ ]:
X = df.drop(columns=['result' ,'symptom6'],axis = 1).values #['gender' ,'result','symptom6']
y = df['result'].values

## splitting into train , validate and test
- random_state is chosen 50 for recreation

In [ ]:
from sklearn.model_selection import train_test_split
X_train ,X_test,y_train,y_test = train_test_split(X,y,shuffle=True,test_size=0.2,random_state=50)

### feautre scaling
 - after testing each model will be scalled indvidually

# importing metrics and validations to be used as required

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay , auc
from sklearn.metrics import roc_curve , RocCurveDisplay
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
score_idx = ["acc","recall","percision","f1 score","AUC"]
comp={}

## important functions

In [ ]:
def scores(y_pred,y_true=y_test):
    res = []
    res +=[accuracy_score(y_true=y_true,y_pred=y_pred)]
    res += [recall_score(y_true=y_true,y_pred=y_pred)]
    res += [precision_score(y_true=y_true,y_pred=y_pred)]
    res += [f1_score(y_true=y_true,y_pred=y_pred)]
    print(f"accuracy score :{res[0]:0.3f} ")
    print(f"recall score :{res[1]:0.3f}     percesion score : {res[2]:0.3f}")
    print(f"f1 score :{res[-1]:0.3f} ")
    return res

In [ ]:
def confmat(y_pred,y_true=y_test):
    cm=confusion_matrix(y_pred=y_pred,y_true=y_true)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=[0,1])
    disp.plot()
    plt.show()

In [ ]:
def rocgraph(y_pred,name,y_test=y_test):
    fpr, tpr, _=roc_curve(y_test,y_pred)
    roc=auc(fpr,tpr)
    display = RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc,estimator_name=name)
    display.plot()
    return (roc)

In [ ]:
#TODO discover the problem ---- > inconsisitent
def kf_validation(model,scoring,n_splits=10):
    kf = KFold(n_splits=n_splits,shuffle=True,random_state=50)
    best_model = None
    best_rec = 0.0
    methods = {"acc":accuracy_score,
               "rec":recall_score,
               "pre":precision_score}
    score = methods[scoring]
    records=[]
    for tr_idx ,test_idx in kf.split(X_train) :

        ## splitting to train and validation
        xtr,xte=X_train[tr_idx],X_train[test_idx]
        ytr,yte=y_train[tr_idx],y_train[test_idx]

        #training
        classifier_k = model
        print(score)
        classifier_k.fit(xtr,ytr)

        #testing
        y_hat = classifier_k.predict(xte)

        #recording and choosing best model
        records+=[score(y_pred=y_hat,y_true=yte)]
        if records[-1] > best_rec :
            best_rec = records[-1]
            best_model = classifier_k
        
    print(f"best {scoring} :{best_rec} ")
    print(f"avg {scoring} : {np.mean(records)}")
    return best_model
    
    

### after searching we found we can do dimensionality reduction to draw a decision boundary

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
def decision_boundary(model) :
    pca = PCA(n_components=2)
    X_test_pca = pca.fit_transform(X_test)
    h = .02  # step size in the mesh
    x_min, x_max = X_test_pca[:, 0].min() - 1, X_test_pca[:, 0].max() + 1
    y_min, y_max = X_test_pca[:, 1].min() - 1, X_test_pca[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))

    # Obtain predictions for each point in the meshgrid
    Z = model.predict(pca.inverse_transform(np.c_[xx.ravel(), yy.ravel()]))

    # Reshape the predictions for contour plotting
    Z = Z.reshape(xx.shape)

    # Plot the decision boundary
    plt.contourf(xx, yy, Z, cmap=plt.cm.RdYlBu, alpha=0.8)

    # Scatter plot of training data points
    plt.scatter(X_test_pca[:, 0], X_test_pca[:, 1], c=y_test, edgecolors='k', cmap=plt.cm.RdYlBu)

    plt.title('Decision Boundary Visualization using PCA')
    plt.xlabel('Principal Component 1')
    plt.ylabel('Principal Component 2')
    plt.show()

# Logestic Regression

### logestic regression reached max iteration with kfolds for validation and without, so we apply standrixation for feautres  and will use grid search to look for best options 

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train_log = sc.fit_transform(X_train)
X_test_log = sc.transform(X_test)
X_val_log = sc.transform(X_validate)

### logestic regression with accuracy as its scoring value without any validation method

In [ ]:
from sklearn.linear_model import LogisticRegression
classifierLR = LogisticRegression()
classifierLR.fit(X_train_log,y_train)

#### test

In [ ]:
y_LR = classifierLR.predict(X_test_log)
confmat(y_LR)
LR_scores = scores(y_LR)

In [ ]:
a= rocgraph(y_LR,'logistic regression')
LR_scores+=[a]
comp['logistic regression']=LR_scores

In [ ]:
decision_boundary(classifierLR)

#### validate

In [ ]:
y_LR = classifierLR.predict(X_val_log)
confmat(y_LR,y_validate)
scores(y_LR,y_validate)

### using k-folds cross validation

In [ ]:
kf = KFold(n_splits=10,shuffle=True,random_state=50)
best_model = None
best_rec = 0.0
recs=[]
for tr_idx ,test_idx in kf.split(X_train_log) :

    xtr,xte=X_train_log[tr_idx],X_train_log[test_idx]
    ytr,yte=y_train[tr_idx],y_train[test_idx]

    classifier_k = LogisticRegression()
    classifier_k.fit(xtr,ytr)
    y_hat = classifier_k.predict(xte)
    recs+=[recall_score(y_pred=y_hat,y_true=yte)]
    if recs[-1] > best_rec :
        best_rec = recs[-1]
        best_model = classifier_k
print("best recall : ", best_rec)
print("avg rec : ", np.mean(recs))
best_model.fit(X_train_log, y_train)
y_hat = best_model.predict(X_test_log)
confmat(y_hat)


In [ ]:
a = rocgraph(y_hat,'logistic regression')
KFold_LR_scores = scores(y_hat)
KFold_LR_scores += [a]
comp['logistic regression k']=KFold_LR_scores

In [ ]:
decision_boundary(best_model)

#### validate

In [ ]:
y_LR = classifierLR.predict(X_val_log)
confmat(y_LR,y_validate)
scores(y_LR,y_validate)

### using GridSearch to find the best paramters to pass into the object creation

### for params we will use : 
 - C : the defintion in they API is :
 > Inverse of regularization strength; must be a positive float. Like in support vector machines, smaller values specify stronger regularization 
  - we already used standrization so we will check if it needs to be multiblied by a number so we will start at 1 to 10000


In [ ]:
param_grid = {
    'C': [1,10,100,1000,10000],
}
best_model = LogisticRegression()
grid_search = GridSearchCV(best_model, param_grid, verbose = 1, cv=10,scoring='recall')


# Fit the model to the training data
grid_search.fit(X_train_log, y_train)

# Get the best parameters
best_params = list(grid_search.best_params_.values())
print(best_params)

In [ ]:
best_model=LogisticRegression(C=best_params[0])
best_model.fit(X_train_log, y_train)
y_hat = best_model.predict(X_test_log)
confmat(y_hat)

In [ ]:
a = rocgraph(y_hat,'logistic regression')
KFold_LR_scores = scores(y_hat)
KFold_LR_scores += [a]
comp['logistic regression grid']=KFold_LR_scores

In [ ]:
decision_boundary(best_model)

#### validate

In [ ]:
y_LR = classifierLR.predict(X_val_log)
confmat(y_LR,y_true=y_validate)
LR_scores = scores(y_LR,y_validate)


# summary

In [ ]:
res_df = pd.DataFrame.from_dict(comp)
res_df.index = score_idx

In [ ]:
res_df

# grid search is much better after trading percision for recall giving a higher AUC and F1 score